In [ ]:
#| default_exp simple_agent.scaler

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace

# Scaler

>  We define a custom scaler to preprocess RL observations

In [ ]:
#| export

import datetime
from typing import Any, Dict
import gym
import torch

import numpy as np
import pandas as pd

from d3rlpy.preprocessing.scalers import Scaler, register_scaler
from betting_env.betting_env import Observation
from betting_agent.datastructure.team_features import *

## Simple scaler

Initially, our RL observations hold certain particular numerical information such as `Opta gameId`, `Opta home team Id`, `Opta away team Id`, the 22 `Opta players Id`, and betting odds (`1X2` and `Asian Handicap`). through this custom scaler, we aim to fetch our `MongoDb` database and extract various features pertaining to both competing teams and the 22 players.

In [ ]:
#| export

class SimpleScaler(Scaler):
    TYPE = "simple"
    OBS = 18
    def __init__(self):
        pass

    def fit_with_env(
        self,
        env: gym.Env,  # Gym environment.
    ):
        "Gets scaling parameters from environment."
        pass

    def transform(
        self,
        x: Observation,  # Observation.
    ) -> np.ndarray:  # Observation features, shape=(1, new_shape), new_shape is the number of features extracted from the database.
        "Returns processed observation."
        
        
        if isinstance(x, Observation):

            # Real-Analytics Home Team Id.
            ra_home_team_id = x.ra_teams_ids[0]

            # Real-Analytics Away Team Id.
            ra_away_team_id = x.ra_teams_ids[1]

            # Game Date.
            game_date = x.game_date

            # 1X2 and Asian Handicap odds.
            if x.numerical_observation.shape[0] != 1:
                x.numerical_observation = (
                    x.numerical_observation.reshape(1, -1)
                )
            odds = x.numerical_observation[0][25:]

            # AH Line.
            ah_line = x.ah_line

            # Extract Home and Away team features
            home_team_feats = self.get_team_features(
                team_id=ra_home_team_id, date=game_date
            )
            away_team_feats = self.get_team_features(
                team_id=ra_away_team_id, date=game_date
            )
            
            # Concatenate all features (Home and Away teams features + 1X2 and AH odds).
            x = np.hstack(
                (home_team_feats, away_team_feats, odds, ah_line)
            ).reshape(1, -1)

        return x

    def reverse_transform(
        self,
        x: torch.Tensor,  # Processed observation.
    ) -> torch.Tensor:
        "Returns reversely transformed observations."
        pass

    def get_params(
        self,
        deep: bool = False,  # Flag to deeply copy objects.
    ) -> Dict[str, Any]:
        "Returns scaling parameters."
        return {}

    def get_team_features(
        self,
        team_id: str,  # Real-analytics team identifier.
        date: datetime.datetime,  # Game Date
    ) -> np.array:  # Team features.
        "Extract teams features from Database."
        feats = TeamFeatures.get_latest(ra_team_id=team_id, date=date)
        team_feats = np.array(
            [feats.elo]  # Elo .
            + [feats.glicko]  # Glicko.
            + [feats.quality_overall]  # team Overall rating.
            + [feats.quality_attack]  # Attack rating.
            + [feats.quality_midfield]  # Midfield rating.
            + [feats.quality_defence]  # Defence rating.
        )

        return team_feats
    
register_scaler(SimpleScaler)

In [ ]:
#| hide
from nbdev import nbdev_export

nbdev_export()